# 🔬 Protein Binding Pocket Finder

This notebook implements a geometry-based algorithm for predicting **ligand binding pockets** in protein structures.

## Principle
Instead of computing expensive energy functions, the protein surface is sampled geometrically:
1. A 3D grid of points is placed over the entire protein
2. Points that are too close to the protein (collision) or too far away (empty space) are removed
3. The remaining points — located inside cavities and pockets — are grouped into clusters
4. Each cluster corresponds to one potential binding pocket

## Input
A `.pdb` file (here: `1H8D.pdb` — a kinase from the RCSB Protein Data Bank)

---

## 0. Imports

All required libraries are imported here in one central place:
- **`numpy`** – fast array operations and vector math
- **`Bio.PDB`** – Biopython module for reading and analyzing PDB files
- **`scipy.spatial.KDTree`** – efficient spatial lookups (much faster than naive distance calculations)
- **`sklearn.cluster.DBSCAN`** – density-based clustering algorithm that requires no fixed cluster count
- **`time`** – runtime measurement

In [9]:
import numpy as np
import time

from Bio import PDB
from scipy.spatial import KDTree
from sklearn.cluster import DBSCAN

---
## 1. Reading the Protein Structure

A PDB file contains the amino acids of the protein as well as water molecules, ions, and ligands.
We only want the **protein atoms** — everything else (HETATM records) is filtered out.

### Why filter?
- **Water** would distort the grid, as water molecules have no fixed position and are often crystallographic artifacts
- **Ligands / co-factors** can obscure the very binding pocket we are trying to find

In PDB files, the first element of the residue ID (`residue.id[0]`) encodes the record type:
- `' '` (space) → standard amino acid ✅
- `'H_...'` → hetero-atom (ligand, ion, etc.) ❌
- `'W'` → water molecule ❌

In [10]:
def get_protein_structure(pdb_file: str):
    """
    Reads a PDB file and returns the structure object along with a list
    of all protein atoms (excluding water and ligands).

    Parameters:
        pdb_file: Path to the .pdb file

    Returns:
        structure:     Bio.PDB Structure object (full hierarchy)
        protein_atoms: List of all Atom objects belonging to amino acids
    """
    # QUIET=True suppresses warnings for minor format inconsistencies
    # PERMISSIVE=True allows non-standard PDB entries
    parser = PDB.PDBParser(QUIET=True, PERMISSIVE=True)

    # Load the structure — 'protein_obj' is an internal label for the object
    structure = parser.get_structure('protein_obj', pdb_file)

    protein_atoms = []

    # Hierarchical iteration: Model → Chain → Residue → Atom
    for model in structure:
        for chain in model:
            for residue in chain:
                # Keep only standard amino acids (no water, no ligands)
                if residue.id[0] == ' ':
                    for atom in residue:
                        protein_atoms.append(atom)

    print(f"✅ Loaded: {len(protein_atoms)} protein atoms found.")
    return structure, protein_atoms


# --- Execution ---
FILE_PATH = "1H8D.pdb"

try:
    structure, atoms = get_protein_structure(FILE_PATH)

    # Spot check: print the first 5 atoms to verify correct parsing
    print("\nCoordinate check (first 5 atoms):")
    for atom in atoms[:5]:
        residue = atom.get_parent()
        print(f"  Residue: {residue.get_resname():>3} | Atom: {atom.get_name():<4} | XYZ: {atom.get_coord()}")

except FileNotFoundError:
    print(f"❌ Error: File '{FILE_PATH}' not found.")

✅ Loaded: 2333 protein atoms found.

Coordinate check (first 5 atoms):
  Residue: ILE | Atom: N    | XYZ: [ 5.169 -8.919 17.688]
  Residue: ILE | Atom: CA   | XYZ: [ 4.5   -8.595 18.976]
  Residue: ILE | Atom: C    | XYZ: [ 3.397 -9.622 19.265]
  Residue: ILE | Atom: O    | XYZ: [ 2.507 -9.808 18.423]
  Residue: ILE | Atom: CB   | XYZ: [ 3.896 -7.17  18.971]


---
## 2. Creating the Search Grid

We place a regular 3D grid over the entire protein — like a chessboard pattern in space, but in three dimensions.

### Parameters:
| Parameter | Meaning | Recommendation |
|-----------|---------|----------------|
| `spacing` | Distance between grid points in Ångström | 2.0 Å (testing), 1.0 Å (final) |
| Buffer (`±5 Å`) | Extra space around the protein | fixed at 5 Å |

### Why `numpy.meshgrid`?
`meshgrid` generates all combinations of x, y, z coordinates at once — much more efficient than three nested `for` loops.

> ⚠️ **Warning:** With `spacing=1.0` the point count explodes (~8× more points than at 2.0 Å). For large proteins this can mean several million points.

In [11]:
def create_search_grid(protein_atoms: list, spacing: float = 2.0) -> np.ndarray:
    """
    Creates a uniform 3D grid around the protein.

    Parameters:
        protein_atoms: List of Biopython Atom objects
        spacing:       Distance between grid points in Ångström

    Returns:
        grid: numpy array of shape (N, 3) containing N grid points
    """
    BUFFER = 5.0  # Å — padding region extending beyond the protein

    # Extract all atom coordinates into a single (N, 3) matrix
    coords = np.array([atom.get_coord() for atom in protein_atoms])

    # Bounding box: smallest and largest coordinates along x, y, z
    min_coords = coords.min(axis=0) - BUFFER
    max_coords = coords.max(axis=0) + BUFFER

    # Generate axis arrays
    x = np.arange(min_coords[0], max_coords[0], spacing)
    y = np.arange(min_coords[1], max_coords[1], spacing)
    z = np.arange(min_coords[2], max_coords[2], spacing)

    # Generate all (x, y, z) combinations and reshape into an (N, 3) matrix
    grid = np.array(np.meshgrid(x, y, z)).T.reshape(-1, 3)

    print(f"📦 Grid created: {grid.shape[0]:,} points at {spacing} Å spacing.")
    return grid


# --- Execution ---
if 'atoms' in locals():
    grid = create_search_grid(atoms, spacing=2.0)

📦 Grid created: 26,796 points at 2.0 Å spacing.


---
## 3. Filtering Pocket Candidates

Not every grid point is meaningful. We apply two geometric filters:

### Filter 1 — Collision check (`min_dist`)
Points **inside** the protein clash with the Van der Waals radii of the atoms.
→ All points with any atom closer than `min_dist` (2.5 Å) are **removed**.

### Filter 2 — Surface proximity (`max_dist`)
Points far **outside** the protein lie in the bulk solvent — no pocket there.
→ Only points with at least one atom within `max_dist` (5.0 Å) are **kept**.

### Why `KDTree`?
A KDTree is a data structure built for spatial lookups. Instead of checking all ~2,333 atoms for each of the ~27,000 grid points (~63 million comparisons), the KDTree reduces the cost to **O(N log N)**.

In [12]:
def find_pocket_points(
    grid_points: np.ndarray,
    protein_atoms: list,
    min_dist: float = 2.5,
    max_dist: float = 5.0
) -> np.ndarray:
    """
    Filters grid points down to potential binding pocket candidates.

    A point qualifies as a pocket candidate if it:
        - does NOT lie within min_dist of any atom (no clash)
        - has at least one atom within max_dist (close to the surface)

    Parameters:
        grid_points:   numpy array (N, 3) of grid points
        protein_atoms: list of Biopython Atom objects
        min_dist:      minimum allowed distance to the protein in Å (Van der Waals buffer)
        max_dist:      maximum allowed distance to the surface in Å

    Returns:
        pocket_points: numpy array (M, 3) of remaining candidate points
    """
    # Extract coordinates from Biopython objects
    atom_coords = np.array([atom.get_coord() for atom in protein_atoms])

    # Build the KDTree once — reused for both filters
    tree = KDTree(atom_coords)

    # --- Filter 1: Collision check ---
    # query_ball_point returns for each grid point all atom indices
    # within min_dist. If the list is empty → no clash → point is kept.
    clash_neighbors = tree.query_ball_point(grid_points, min_dist)
    no_clash_mask = np.array([len(neighbors) == 0 for neighbors in clash_neighbors])
    candidates = grid_points[no_clash_mask]

    # --- Filter 2: Surface proximity ---
    # From the candidate set: keep only points that have at least one atom
    # within max_dist (i.e. close to the protein surface)
    surface_neighbors = tree.query_ball_point(candidates, max_dist)
    near_surface_mask = np.array([len(neighbors) > 0 for neighbors in surface_neighbors])
    pocket_points = candidates[near_surface_mask]

    print(f"🔍 Filtered: {len(grid_points):,} → {len(pocket_points):,} pocket candidates "
          f"({len(pocket_points)/len(grid_points)*100:.1f}% remaining).")
    return pocket_points


# --- Execution with runtime measurement ---
if 'atoms' in locals() and 'grid' in locals():
    start = time.time()
    pocket_candidates = find_pocket_points(grid, atoms)
    elapsed = time.time() - start

🔍 Filtered: 26,796 → 4,087 pocket candidates (15.3% remaining).


---
## 4. Cluster Analysis: Identifying Individual Pockets

The ~4,000 pocket points form connected regions in space. We use **DBSCAN** to group them into discrete pockets.

### Why DBSCAN instead of k-Means?
- DBSCAN requires **no prior specification** of the number of clusters
- DBSCAN detects clusters of arbitrary shape (pockets are irregular)
- DBSCAN marks isolated outlier points as **noise** (label `-1`)

### Parameters:
| Parameter | Meaning | Typical value |
|-----------|---------|---------------|
| `eps` | Maximum distance between two points in the same cluster | 2.5 Å |
| `min_samples` | Minimum points required to form a cluster | 5 |

Note on Filtering: By setting min_samples = 5, we ensure that tiny surface indentations (1–4 points) are ignored. Each grid point at 2.0 Å spacing represents a volume of 8.0 Å³. A cluster of 5 points represents ~40 Å³, which is the minimum size required for even the smallest fragments to bind.

In [13]:
def cluster_pocket_points(
    pocket_points: np.ndarray,
    eps: float = 2.5,
    min_samples: int = 5
) -> dict:
    """
    Groups pocket points into discrete binding pockets using DBSCAN.
    Groups smaller than 'min_samples' are automatically discarded as noise.
    """
    if len(pocket_points) == 0:
        return {}

    # 1. Run DBSCAN: The 'min_samples' parameter filters out small clusters automatically
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(pocket_points)
    labels = db.labels_ 

    # 2. Extract clusters (exclude -1 which is the noise label)
    unique_labels = set(labels) - {-1}
    raw_pockets = {
        label: pocket_points[labels == label]
        for label in unique_labels
    }

    # 3. Sort by size (largest first) and re-index for cleaner output (0, 1, 2...)
    sorted_pockets_list = sorted(raw_pockets.values(), key=len, reverse=True)
    pockets = {i: points for i, points in enumerate(sorted_pockets_list)}

    # Summary Output
    noise_count = np.sum(labels == -1)
    print(f"🔵 Clustering complete: {len(pockets)} pockets found ({noise_count} noise points discarded).")
    
    if pockets:
        top_pocket_size = len(pockets[0])
        print(f"✅ Rank 1 Pocket (Largest): {top_pocket_size} points.")
        
    return pockets

---
## 5. Export for Visualization in UCSF Chimera / ChimeraX

Each pocket is saved as a separate `.pdb` file, with every point encoded as a dummy `HETATM` atom.
All files can then be loaded simultaneously in Chimera and color-coded individually.

**Workflow in Chimera:**
1. Load the protein: `File → Open → 1H8D.pdb`
2. Load pockets: `File → Open → step3_pocket_0.pdb`, etc.
3. Set representation: `Actions → Atoms/Bonds → sphere`
4. Set colors: `Actions → Color → ...`

In [14]:
def save_points_to_pdb(points: np.ndarray, output_file: str) -> None:
    """
    Saves a numpy array of 3D coordinates as a PDB file.
    Each point is written as a HETATM entry (dummy atom 'N').

    Parameters:
        points:      numpy array (N, 3) with XYZ coordinates
        output_file: target file path
    """
    with open(output_file, 'w') as f:
        for i, (x, y, z) in enumerate(points):
            # PDB HETATM format (column-accurate per PDB specification):
            # Cols 1-6: Record type | 7-11: Atom serial | 13-16: Atom name
            # 18-20: Residue name | 22: Chain | 23-26: Residue seq | 31-54: X, Y, Z
            f.write(f"HETATM{i+1:5d}  P   PTS A   1    {x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           N\n")
        f.write("END\n")
    print(f"   💾 Saved: {output_file} ({len(points)} points)")


def export_all_steps(atoms: list, output_dir: str = ".") -> None:
    """
    Runs the full pipeline and exports each stage as a PDB file.

    Output files:
        step1_full_grid.pdb          — the complete grid around the protein
        step2_pocket_candidates.pdb  — filtered pocket candidate points
        step3_pocket_N.pdb           — individual clustered pockets (sorted by size)
    """
    import os
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Full grid
    print("📦 Step 1: Generating grid ...")
    full_grid = create_search_grid(atoms, spacing=2.0)
    save_points_to_pdb(full_grid, f"{output_dir}/step1_full_grid.pdb")

    # Step 2: Filtered candidates
    print("\n🔍 Step 2: Filtering candidates ...")
    candidates = find_pocket_points(full_grid, atoms)
    save_points_to_pdb(candidates, f"{output_dir}/step2_pocket_candidates.pdb")

    # Step 3: Clustering
    print("\nStep 3: Clustering pockets ...")
    pockets = cluster_pocket_points(candidates)

    print("\nxporting pockets ...")
    for rank, (p_id, p_points) in enumerate(pockets.items()):
        save_points_to_pdb(p_points, f"{output_dir}/step3_pocket_{rank}.pdb")

    print(f"\nExport complete. {len(pockets)} pocket files saved.")


# --- Execution ---
if 'atoms' in locals():
    export_all_steps(atoms, output_dir="pocket_output")

📦 Step 1: Generating grid ...
📦 Grid created: 26,796 points at 2.0 Å spacing.
   💾 Saved: pocket_output/step1_full_grid.pdb (26796 points)

🔍 Step 2: Filtering candidates ...
🔍 Filtered: 26,796 → 4,087 pocket candidates (15.3% remaining).
   💾 Saved: pocket_output/step2_pocket_candidates.pdb (4087 points)

Step 3: Clustering pockets ...
🔵 Clustering complete: 22 pockets found (126 noise points discarded).
✅ Rank 1 Pocket (Largest): 3925 points.

xporting pockets ...
   💾 Saved: pocket_output/step3_pocket_0.pdb (3925 points)
   💾 Saved: pocket_output/step3_pocket_1.pdb (5 points)
   💾 Saved: pocket_output/step3_pocket_2.pdb (4 points)
   💾 Saved: pocket_output/step3_pocket_3.pdb (3 points)
   💾 Saved: pocket_output/step3_pocket_4.pdb (3 points)
   💾 Saved: pocket_output/step3_pocket_5.pdb (2 points)
   💾 Saved: pocket_output/step3_pocket_6.pdb (2 points)
   💾 Saved: pocket_output/step3_pocket_7.pdb (2 points)
   💾 Saved: pocket_output/step3_pocket_8.pdb (2 points)
   💾 Saved: pocket_out

---
## 💡 Potential Improvements

### Short-term (easy to implement)

**1. Rank pockets by volume instead of point count**
The largest pocket is not always the biologically most relevant one. Volume can be estimated as:
```python
volume_A3 = len(pocket_points) * spacing**3
```

**2. Adapt `min_dist` to per-element Van der Waals radii**
Currently `min_dist=2.5 Å` is uniform for all atoms. A more realistic approach:
```python
VDW_RADII = {'C': 1.7, 'N': 1.55, 'O': 1.52, 'S': 1.8}
```

**3. Configuration file / argparse**
Parameters like `spacing`, `min_dist`, `max_dist` should not be hardcoded.
They should be passed via CLI arguments or a `.yaml` config file.

---

### Mid-term (higher impact)

**4. Use only model 0**
NMR structures in PDB files contain multiple models (conformers).
Currently the code iterates over all models and accumulates atoms from all of them. Better:
```python
model = structure[0]  # Use only the first model
```

**5. Use B-factors as a flexibility filter**
Atoms with a high B-factor (> 60) are highly flexible and do not define a stable pocket.
These atoms could be excluded from the KDTree.

**6. Compute a depth score (SAS-based)**
A point surrounded by many atoms lies deeper inside a pocket.
This can be measured by the number of neighbors in the KDTree:
```python
depth_score = tree.query_ball_point(point, max_dist, return_length=True)
```

**7. Validate against the known binding site**
For `1H8D` the real ligand is known. You could check whether Pocket #0 actually
overlaps with the ligand coordinates from the PDB — that would be a real quality test.

---

### Long-term (research project)

**8. Integrate additional scoring functions**
- Hydrophobicity of surrounding residues (Ala, Val, Leu, Ile → high score)
- Conservation of pocket residues (via MSA / ConSurf)
- Benchmarking against `fpocket`, `SiteMap`, or `AutoSite`

**9. In-notebook visualization with py3Dmol**
```python
import py3Dmol
viewer = py3Dmol.view()
viewer.addModel(open('1H8D.pdb').read(), 'pdb')
viewer.setStyle({'cartoon': {'color': 'spectrum'}})
viewer.show()
```